# __텍스트 분류 모델 만들기__

**분류 작업 이해하기** 
--- 
👉 분류 작업은 목표값(Target)이 속한 범주(Category 또는 Class)를 예측하기 위해 사용하는 머신러닝(기계학습/Machine Learning)의 한 형태입니다. 예를 들어, 남성 또는 여성을 분류하는 이진 분류와 동물의 종(개, 고양이, 토끼 등)을 예측하는 다중 분류 모두 분류 작업에 포함됩니다.

**자연어 처리(NLP, Natural Language Processing)란** 
--- 
NLP는 작업의 목적에 따라 자연어 이해(NLU, Natural Language Understanding)와 자연어 생성(NLG, Natural Language Generation)으로 분류할 수 있습니다. 자연어 이해는 사람이 이해하는 자연어를 컴퓨터가 이해할 수 있는 값으로 바꾸는 처리를 의미합니다. 반면에 자연어 생성은 더 나아가 컴퓨터가 이해할 수 있는 값을 사람이 이해하도록 자연어로 바꾸는 과정을 의미합니다.

**본 튜토리얼에서는**
--- 
👉 대표적인 머신러닝 경진대회 플랫폼인 캐글의 IMDB Movie Reviews 데이터 세트를 사용하여 영화 리뷰의 감정을 분류하는 모델을 만듭니다. 이 데이터 세트는 50,000 개의 영화 리뷰 텍스트와 긍정 또는 부정 감정에 대한 목표값(Target)으로 구성되어 있습니다. 영화 평점을 기준으로 5보다 작은 값을 부정, 7보다 큰 값을 긍정으로 표현하였으며 각 개별 영화는 30 개 이상의 리뷰 결과를 갖지 않습니다.

__튜토리얼 주의사항__    
> - 텍스트 분류 모델은 하나의 텍스트에서 하나의 목표값(Target, 범주)를 예측하는 용도로 사용할 수 있습니다.
> - 텍스트를 나타내는 컬럼과, 텍스트의 목표값을 나타내는 컬럼이 존재해야 합니다.
> - 해당 텍스트 분류 모델의 베이스 모델(`ELECTRA`)은 GPU를 사용합니다. 사용한 모델의 크기와 배치 사이즈에 따라 GPU 메모리가 부족할 수 있습니다. 이 경우, 더 작은 모델을 사용하시거나 배치 사이즈를 줄여보십시오.

## __0. 데이터 준비하기__

ThanoSQL의 쿼리 구문을 사용하기 위해서는 [ThanoSQL 워크스페이스](https://docs.thanosql.ai/quick_start/how_to_use_ThanoSQL/#5-thanosql)
에서 언급된 것처럼 API 토큰을 생성하고 아래의 쿼리를 실행해야 합니다.   

In [ ]:
%load_ext thanosql
%thanosql API_TOKEN=<발급받은 개인 토큰>

In [2]:
%%thanosql
COPY movie_review_train FROM "tutorial_data/movie_review_data/movie_review_train.csv"

Success


In [3]:
%%thanosql
COPY movie_review_test FROM "tutorial_data/movie_review_data/movie_review_test.csv"

Success


## __1. 데이터 세트 확인__

영화 리뷰 감정 분류 모델을 만들기 위해 우리는 ThanoSQL DB에 저장되어 있는 movie_review_train 테이블을 사용합니다. 아래의 쿼리문을 실행하여 테이블 내용을 확인합니다.

In [4]:
%%thanosql
SELECT *
FROM movie_review_train
LIMIT 5

,review,sentiment
0,This is the kind of movie that BEGS to be show...,negative
1,Bulletproof is quite clearly a disposable film...,negative
2,A beautiful shopgirl in London is swept off he...,positive
3,"VERY dull, obvious, tedious Exorcist rip-off f...",negative
4,Do we really need any more narcissistic garbag...,negative


__데이터 이해하기__ 
- review : 영화 리뷰 텍스트 
- sentiment : 해당 리뷰가 긍정적인지(positive), 부정적인지(negative)를 나타내는 목표값(Target)

## __2. 사전 학습된 모델을 사용하여 영화 리뷰 감정 분류 결과 예측__

먼저 저희가 사전에 학습해둔 모델로 바로 결과를 예측해보겠습니다. 다음 쿼리문을 실행하면 tutorial_text_classification 모델을 사용하여 영화 리뷰 분류 결과를 예측해볼 수 있습니다.

In [5]:
%%thanosql
PREDICT USING tutorial_text_classification
OPTIONS (
    text_col='review',
    label_col='sentiment'
    )
AS
SELECT *
FROM movie_review_test

,review,sentiment,predicted
0,"I read the book before seeing the movie, and t...",positive,positive
1,"""9/11,"" hosted by Robert DeNiro, presents foot...",positive,positive
2,"Yesterday I attended the world premiere of ""De...",positive,positive
3,Moonwalker is a Fantasy Music film staring Mic...,positive,positive
4,"Welcome to Oakland, where the dead come out to...",positive,positive
...,...,...,...
995,I remember catching this movie on one of the S...,negative,negative
996,CyberTracker is set in Los Angeles sometime in...,negative,negative
997,"There is so much that is wrong with this film,...",negative,negative
998,"I am a firm believer that a film, TV serial or...",positive,positive


## __3. 텍스트 분류 모델 만들기__

이전 단계에서 확인한 movie_review_train 데이터 세트를 사용하여 텍스트 분류 모델을 만듭니다. 아래의 쿼리 구문을 실행하여 my_movie_review_classifier라는 이름의 모델을 만듭니다.

(쿼리 실행 시 예상 소요 시간 : 3 min)

In [6]:
%%thanosql
BUILD MODEL my_movie_review_classifier
USING ElectraEn
OPTIONS (
    text_col='review',
    label_col='sentiment',
    epochs=1,
    batch_size=4
    )
AS
SELECT *
FROM movie_review_train

Success


__쿼리 세부 정보__ 
- "__BUILD MODEL__" 쿼리 구문을 사용하여 <mark style="background-color:#E9D7FD ">my_movie_review_classifier</mark>라는 모델을 만들고 학습시킵니다. 
- "__USING__" 쿼리 구문을 통해 베이스 모델로 `ElectraEn`을 사용할 것을 명시합니다. 
- "__OPTIONS__" 쿼리 구문을 통해 모델 생성에 사용할 옵션을 지정합니다. "text_col"은 학습에 사용할 텍스트를 담은 컬럼의 이름이며, "label_col"은 목표값의 정보를 담은 컬럼의 이름입니다. "epochs"를 통해 몇 번의 학습을 반복할 지를 지정합니다. "batch_size"는 한 번의 학습에서 읽는 데이터 세트 묶음의 크기입니다.

## __4. 생성된 모델을 사용하여 영화 리뷰 감정 분류 결과 예측__

이전 단계에서 만든 텍스트 분류 예측 모델을 사용해서 특정 리뷰(학습에 이용되지 않은 데이터 테이블, movie_review_test)의 목표값을 예측해 봅니다.

In [7]:
%%thanosql
PREDICT USING my_movie_review_classifier
OPTIONS (
    text_col='review'
    )
AS
SELECT *
FROM movie_review_test

,review,sentiment,predicted
0,"I read the book before seeing the movie, and t...",positive,positive
1,"""9/11,"" hosted by Robert DeNiro, presents foot...",positive,positive
2,"Yesterday I attended the world premiere of ""De...",positive,positive
3,Moonwalker is a Fantasy Music film staring Mic...,positive,positive
4,"Welcome to Oakland, where the dead come out to...",positive,negative
...,...,...,...
995,I remember catching this movie on one of the S...,negative,negative
996,CyberTracker is set in Los Angeles sometime in...,negative,negative
997,"There is so much that is wrong with this film,...",negative,negative
998,"I am a firm believer that a film, TV serial or...",positive,positive


__쿼리 세부 정보__ 
- "PREDICT USING" 쿼리 구문을 통해 이전 단계에서 만든 my_movie_review_classifier 모델을 예측에 사용합니다. 
- "OPTIONS"를 통해 예측에 사용할 옵션을 지정합니다. review는 예측에 사용할 텍스트를 담은 컬럼의 이름입니다. 예측 결과는 predicted 컬럼에 저장되어 반환됩니다.